# Step 3: 다층 퍼셉트론 (MLP) - 딥러닝의 시작

이제 진짜 딥러닝의 세계로 들어갑니다! 다층 퍼셉트론(Multi-Layer Perceptron)과 역전파(Backpropagation) 알고리즘을 직접 구현해봅시다.

## 학습 목표
1. 순전파(Forward Propagation) 이해하고 구현하기
2. 역전파(Backpropagation) 알고리즘 이해하기
3. 경사하강법(Gradient Descent)으로 학습하기
4. 실제 분류 문제 해결하기

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons, make_circles
from sklearn.model_selection import train_test_split

# 시각화 설정
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False
np.random.seed(42)

## 1. 다층 퍼셉트론의 구조

다층 퍼셉트론은 입력층, 은닉층(들), 출력층으로 구성됩니다.

### 순전파 과정:
1. 입력층 → 은닉층: $h = f(W_1 \cdot x + b_1)$
2. 은닉층 → 출력층: $y = f(W_2 \cdot h + b_2)$

여기서 $f$는 활성화 함수입니다.

In [ ]:
# 활성화 함수와 그 미분
def sigmoid(x):
    """시그모이드 함수"""
    return 1 / (1 + np.exp(-np.clip(x, -500, 500)))  # 오버플로우 방지

def sigmoid_derivative(x):
    """시그모이드 함수의 미분"""
    return x * (1 - x)

def relu(x):
    """ReLU 함수"""
    return np.maximum(0, x)

def relu_derivative(x):
    """ReLU 함수의 미분"""
    return np.where(x > 0, 1, 0)

def tanh(x):
    """Tanh 함수"""
    return np.tanh(x)

def tanh_derivative(x):
    """Tanh 함수의 미분"""
    return 1 - x**2

# 활성화 함수 시각화
x = np.linspace(-5, 5, 100)
plt.figure(figsize=(12, 4))

# 함수값
plt.subplot(1, 2, 1)
plt.plot(x, sigmoid(x), label='Sigmoid', linewidth=2)
plt.plot(x, relu(x), label='ReLU', linewidth=2)
plt.plot(x, tanh(x), label='Tanh', linewidth=2)
plt.xlabel('x')
plt.ylabel('f(x)')
plt.title('Activation Functions')
plt.legend()
plt.grid(True, alpha=0.3)

# 미분값
plt.subplot(1, 2, 2)
plt.plot(x, sigmoid_derivative(sigmoid(x)), label="Sigmoid'", linewidth=2)
plt.plot(x, relu_derivative(x), label="ReLU'", linewidth=2)
plt.plot(x, tanh_derivative(tanh(x)), label="Tanh'", linewidth=2)
plt.xlabel('x')
plt.ylabel("f'(x)")
plt.title('Derivatives of Activation Functions')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. MLP 클래스 구현

이제 다층 퍼셉트론을 처음부터 구현해봅시다!

In [ ]:
class MLP:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.1, activation='sigmoid'):
        """
        다층 퍼셉트론 초기화
        
        Parameters:
        - input_size: 입력층 크기
        - hidden_size: 은닉층 크기
        - output_size: 출력층 크기
        - learning_rate: 학습률
        - activation: 활성화 함수 ('sigmoid', 'relu', 'tanh')
        """
        # 가중치 초기화 (Xavier 초기화)
        self.W1 = np.random.randn(input_size, hidden_size) * np.sqrt(1 / input_size)
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * np.sqrt(1 / hidden_size)
        self.b2 = np.zeros((1, output_size))
        
        self.learning_rate = learning_rate
        
        # 활성화 함수 설정
        if activation == 'sigmoid':
            self.activation = sigmoid
            self.activation_derivative = sigmoid_derivative
        elif activation == 'relu':
            self.activation = relu
            self.activation_derivative = relu_derivative
        elif activation == 'tanh':
            self.activation = tanh
            self.activation_derivative = tanh_derivative
        
        # 학습 과정 기록
        self.losses = []
    
    def forward(self, X):
        """
        순전파
        """
        # 입력층 → 은닉층
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = self.activation(self.z1)
        
        # 은닉층 → 출력층
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = sigmoid(self.z2)  # 출력층은 항상 시그모이드 (이진 분류)
        
        return self.a2
    
    def backward(self, X, y, output):
        """
        역전파
        """
        m = X.shape[0]  # 샘플 수
        
        # 출력층 오차
        self.dz2 = output - y
        self.dW2 = (1/m) * np.dot(self.a1.T, self.dz2)
        self.db2 = (1/m) * np.sum(self.dz2, axis=0, keepdims=True)
        
        # 은닉층 오차
        da1 = np.dot(self.dz2, self.W2.T)
        self.dz1 = da1 * self.activation_derivative(self.a1)
        self.dW1 = (1/m) * np.dot(X.T, self.dz1)
        self.db1 = (1/m) * np.sum(self.dz1, axis=0, keepdims=True)
        
        # 가중치 업데이트
        self.W2 -= self.learning_rate * self.dW2
        self.b2 -= self.learning_rate * self.db2
        self.W1 -= self.learning_rate * self.dW1
        self.b1 -= self.learning_rate * self.db1
    
    def train(self, X, y, epochs=1000, verbose=True):
        """
        모델 학습
        """
        for epoch in range(epochs):
            # 순전파
            output = self.forward(X)
            
            # 손실 계산 (Binary Cross-Entropy)
            loss = -np.mean(y * np.log(output + 1e-8) + (1 - y) * np.log(1 - output + 1e-8))
            self.losses.append(loss)
            
            # 역전파
            self.backward(X, y, output)
            
            # 진행상황 출력
            if verbose and epoch % 100 == 0:
                accuracy = self.accuracy(X, y)
                print(f"Epoch {epoch}, Loss: {loss:.4f}, Accuracy: {accuracy:.2%}")
    
    def predict(self, X):
        """
        예측
        """
        output = self.forward(X)
        return (output > 0.5).astype(int)
    
    def accuracy(self, X, y):
        """
        정확도 계산
        """
        predictions = self.predict(X)
        return np.mean(predictions == y)

## 3. XOR 문제 해결하기

이제 퍼셉트론으로는 해결할 수 없었던 XOR 문제를 해결해봅시다!

In [ ]:
# XOR 데이터
X_xor = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y_xor = np.array([[0], [1], [1], [0]])

# MLP 모델 생성 및 학습
mlp_xor = MLP(input_size=2, hidden_size=4, output_size=1, learning_rate=0.5)
mlp_xor.train(X_xor, y_xor, epochs=5000, verbose=False)

# 결과 시각화
plt.figure(figsize=(12, 4))

# 학습 곡선
plt.subplot(1, 3, 1)
plt.plot(mlp_xor.losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.grid(True, alpha=0.3)

# 결정 경계
plt.subplot(1, 3, 2)
x_min, x_max = -0.5, 1.5
y_min, y_max = -0.5, 1.5
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                     np.linspace(y_min, y_max, 100))
Z = mlp_xor.forward(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

plt.contourf(xx, yy, Z, levels=20, alpha=0.8, cmap='RdBu')
plt.colorbar(label='Output')
plt.scatter(X_xor[y_xor.ravel() == 0][:, 0], X_xor[y_xor.ravel() == 0][:, 1], 
            c='red', s=200, edgecolors='black', linewidths=2, label='0')
plt.scatter(X_xor[y_xor.ravel() == 1][:, 0], X_xor[y_xor.ravel() == 1][:, 1], 
            c='blue', s=200, edgecolors='black', linewidths=2, label='1')
plt.xlabel('x1')
plt.ylabel('x2')
plt.title('XOR Decision Boundary')
plt.legend()
plt.grid(True, alpha=0.3)

# 예측 결과
plt.subplot(1, 3, 3)
predictions = mlp_xor.predict(X_xor)
output_probs = mlp_xor.forward(X_xor)
table_data = [[x1, x2, y[0], pred[0], f"{prob[0]:.3f}"] 
              for (x1, x2), y, pred, prob in zip(X_xor, y_xor, predictions, output_probs)]
plt.table(cellText=table_data,
          colLabels=['x1', 'x2', 'Target', 'Prediction', 'Probability'],
          cellLoc='center',
          loc='center')
plt.axis('off')
plt.title('XOR Predictions')

plt.tight_layout()
plt.show()

print(f"최종 정확도: {mlp_xor.accuracy(X_xor, y_xor):.2%}")

## 4. 더 복잡한 데이터셋으로 실험

이제 더 복잡한 비선형 패턴을 가진 데이터를 분류해봅시다.

In [ ]:
# 다양한 데이터셋 생성
datasets = {
    'moons': make_moons(n_samples=200, noise=0.2, random_state=42),
    'circles': make_circles(n_samples=200, noise=0.2, factor=0.5, random_state=42)
}

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

for idx, (name, (X, y)) in enumerate(datasets.items()):
    # 데이터 정규화
    X = (X - X.mean(axis=0)) / X.std(axis=0)
    y = y.reshape(-1, 1)
    
    # 학습/테스트 분할
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # 모델 학습
    mlp = MLP(input_size=2, hidden_size=10, output_size=1, learning_rate=0.5, activation='tanh')
    mlp.train(X_train, y_train, epochs=1000, verbose=False)
    
    # 원본 데이터 시각화
    ax = axes[idx, 0]
    ax.scatter(X[y.ravel() == 0][:, 0], X[y.ravel() == 0][:, 1], c='red', alpha=0.6, label='Class 0')
    ax.scatter(X[y.ravel() == 1][:, 0], X[y.ravel() == 1][:, 1], c='blue', alpha=0.6, label='Class 1')
    ax.set_title(f'{name.capitalize()} Dataset')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # 결정 경계
    ax = axes[idx, 1]
    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                         np.linspace(y_min, y_max, 100))
    Z = mlp.forward(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    ax.contourf(xx, yy, Z, levels=20, alpha=0.8, cmap='RdBu')
    ax.scatter(X[y.ravel() == 0][:, 0], X[y.ravel() == 0][:, 1], c='red', edgecolors='black', linewidths=1)
    ax.scatter(X[y.ravel() == 1][:, 0], X[y.ravel() == 1][:, 1], c='blue', edgecolors='black', linewidths=1)
    ax.set_title('Decision Boundary')
    ax.grid(True, alpha=0.3)
    
    # 학습 곡선
    ax = axes[idx, 2]
    ax.plot(mlp.losses)
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss')
    ax.set_title('Training Loss')
    ax.grid(True, alpha=0.3)
    
    # 정확도 출력
    train_acc = mlp.accuracy(X_train, y_train)
    test_acc = mlp.accuracy(X_test, y_test)
    print(f"{name.capitalize()} - Train Accuracy: {train_acc:.2%}, Test Accuracy: {test_acc:.2%}")

plt.tight_layout()
plt.show()

## 5. 은닉층 뉴런 수의 영향

은닉층의 뉴런 수가 모델의 표현력에 어떤 영향을 미치는지 살펴봅시다.

In [ ]:
# Moons 데이터셋 사용
X, y = make_moons(n_samples=200, noise=0.2, random_state=42)
X = (X - X.mean(axis=0)) / X.std(axis=0)
y = y.reshape(-1, 1)

# 다양한 은닉층 크기 실험
hidden_sizes = [1, 2, 5, 10, 20, 50]
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

for idx, hidden_size in enumerate(hidden_sizes):
    # 모델 학습
    mlp = MLP(input_size=2, hidden_size=hidden_size, output_size=1, learning_rate=0.5)
    mlp.train(X, y, epochs=1000, verbose=False)
    
    # 결정 경계 시각화
    ax = axes[idx]
    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                         np.linspace(y_min, y_max, 100))
    Z = mlp.forward(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    ax.contourf(xx, yy, Z, levels=20, alpha=0.8, cmap='RdBu')
    ax.scatter(X[y.ravel() == 0][:, 0], X[y.ravel() == 0][:, 1], c='red', edgecolors='black', linewidths=1)
    ax.scatter(X[y.ravel() == 1][:, 0], X[y.ravel() == 1][:, 1], c='blue', edgecolors='black', linewidths=1)
    
    accuracy = mlp.accuracy(X, y)
    ax.set_title(f'Hidden Size: {hidden_size}\nAccuracy: {accuracy:.2%}')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. 가중치 시각화

학습된 가중치가 어떻게 생겼는지 살펴봅시다.

In [ ]:
# 간단한 2-4-1 네트워크로 XOR 학습
mlp_vis = MLP(input_size=2, hidden_size=4, output_size=1, learning_rate=0.5)
mlp_vis.train(X_xor, y_xor, epochs=5000, verbose=False)

# 가중치 시각화
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# W1 가중치 행렬
ax = axes[0]
im1 = ax.imshow(mlp_vis.W1, cmap='RdBu', aspect='auto')
ax.set_title('W1 (Input → Hidden)')
ax.set_xlabel('Hidden Neurons')
ax.set_ylabel('Input Features')
ax.set_xticks(range(4))
ax.set_yticks(range(2))
ax.set_yticklabels(['x1', 'x2'])
plt.colorbar(im1, ax=ax)

# 각 은닉 뉴런의 가중치 값
for i in range(2):
    for j in range(4):
        ax.text(j, i, f'{mlp_vis.W1[i, j]:.2f}', ha='center', va='center')

# W2 가중치 행렬
ax = axes[1]
im2 = ax.imshow(mlp_vis.W2.T, cmap='RdBu', aspect='auto')
ax.set_title('W2 (Hidden → Output)')
ax.set_xlabel('Hidden Neurons')
ax.set_ylabel('Output')
ax.set_xticks(range(4))
ax.set_yticks([0])
ax.set_yticklabels(['y'])
plt.colorbar(im2, ax=ax)

# 각 가중치 값
for j in range(4):
    ax.text(j, 0, f'{mlp_vis.W2[j, 0]:.2f}', ha='center', va='center')

# 네트워크 다이어그램
ax = axes[2]
ax.set_title('Network Architecture')

# 노드 위치
layers = [2, 4, 1]
layer_positions = [0, 1, 2]
colors = ['lightblue', 'lightgreen', 'lightcoral']

# 노드와 연결선 그리기
for l_idx, (layer_size, x_pos, color) in enumerate(zip(layers, layer_positions, colors)):
    y_positions = np.linspace(0, 1, layer_size)
    
    # 노드 그리기
    for y_pos in y_positions:
        ax.scatter(x_pos, y_pos, s=500, c=color, edgecolors='black', linewidths=2, zorder=3)
    
    # 연결선 그리기
    if l_idx < len(layers) - 1:
        next_layer_size = layers[l_idx + 1]
        next_y_positions = np.linspace(0, 1, next_layer_size)
        
        for y1 in y_positions:
            for y2 in next_y_positions:
                ax.plot([x_pos, layer_positions[l_idx + 1]], [y1, y2], 
                       'k-', alpha=0.3, linewidth=1)

ax.set_xlim(-0.5, 2.5)
ax.set_ylim(-0.2, 1.2)
ax.axis('off')

# 레이어 라벨
ax.text(0, -0.1, 'Input\nLayer', ha='center', fontsize=10)
ax.text(1, -0.1, 'Hidden\nLayer', ha='center', fontsize=10)
ax.text(2, -0.1, 'Output\nLayer', ha='center', fontsize=10)

plt.tight_layout()
plt.show()

## 7. 미니배치 학습 구현

실제 딥러닝에서는 전체 데이터를 한 번에 학습하지 않고, 작은 배치로 나누어 학습합니다.

In [ ]:
class MLPWithMiniBatch(MLP):
    def train_with_batches(self, X, y, epochs=1000, batch_size=32, verbose=True):
        """
        미니배치를 사용한 학습
        """
        n_samples = X.shape[0]
        n_batches = n_samples // batch_size
        
        for epoch in range(epochs):
            # 데이터 셔플
            indices = np.random.permutation(n_samples)
            X_shuffled = X[indices]
            y_shuffled = y[indices]
            
            epoch_loss = 0
            
            # 미니배치 학습
            for i in range(n_batches):
                start_idx = i * batch_size
                end_idx = start_idx + batch_size
                
                X_batch = X_shuffled[start_idx:end_idx]
                y_batch = y_shuffled[start_idx:end_idx]
                
                # 순전파
                output = self.forward(X_batch)
                
                # 손실 계산
                batch_loss = -np.mean(y_batch * np.log(output + 1e-8) + 
                                     (1 - y_batch) * np.log(1 - output + 1e-8))
                epoch_loss += batch_loss
                
                # 역전파
                self.backward(X_batch, y_batch, output)
            
            # 평균 손실
            epoch_loss /= n_batches
            self.losses.append(epoch_loss)
            
            # 진행상황 출력
            if verbose and epoch % 100 == 0:
                accuracy = self.accuracy(X, y)
                print(f"Epoch {epoch}, Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2%}")

# 더 큰 데이터셋으로 테스트
X_large, y_large = make_moons(n_samples=1000, noise=0.2, random_state=42)
X_large = (X_large - X_large.mean(axis=0)) / X_large.std(axis=0)
y_large = y_large.reshape(-1, 1)

# 전체 배치 vs 미니배치 비교
mlp_full = MLPWithMiniBatch(input_size=2, hidden_size=10, output_size=1, learning_rate=0.5)
mlp_mini = MLPWithMiniBatch(input_size=2, hidden_size=10, output_size=1, learning_rate=0.5)

print("전체 배치 학습:")
mlp_full.train(X_large, y_large, epochs=500, verbose=False)

print("\n미니배치 학습:")
mlp_mini.train_with_batches(X_large, y_large, epochs=500, batch_size=32, verbose=False)

# 학습 곡선 비교
plt.figure(figsize=(10, 6))
plt.plot(mlp_full.losses, label='Full Batch', linewidth=2)
plt.plot(mlp_mini.losses, label='Mini-Batch (size=32)', linewidth=2, alpha=0.8)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Full Batch vs Mini-Batch Training')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"\n최종 정확도 - 전체 배치: {mlp_full.accuracy(X_large, y_large):.2%}")
print(f"최종 정확도 - 미니배치: {mlp_mini.accuracy(X_large, y_large):.2%}")

## 8. 연습 문제

In [ ]:
# 문제 1: 드롭아웃(Dropout) 구현
# 과적합을 방지하는 정규화 기법
class MLPWithDropout(MLP):
    def __init__(self, *args, dropout_rate=0.5, **kwargs):
        super().__init__(*args, **kwargs)
        self.dropout_rate = dropout_rate
    
    def forward_with_dropout(self, X, training=True):
        # 힌트: training=True일 때만 드롭아웃 적용
        # np.random.binomial(1, 1-self.dropout_rate, size=...) 사용
        pass

# 문제 2: 다중 클래스 분류를 위한 소프트맥스 출력층 구현
def softmax(x):
    # 힌트: exp(x) / sum(exp(x), axis=1)
    # 수치 안정성을 위해 x - max(x) 먼저 계산
    pass

# 문제 3: 학습률 감소(Learning Rate Decay) 구현
# 에폭이 진행될수록 학습률을 감소시키는 기법
def learning_rate_decay(initial_lr, epoch, decay_rate=0.99):
    # 힌트: initial_lr * (decay_rate ** epoch)
    pass

## 정리

이번 튜토리얼에서 배운 내용:
1. 다층 퍼셉트론(MLP)의 구조와 순전파
2. 역전파 알고리즘의 원리와 구현
3. 경사하강법을 통한 가중치 학습
4. 비선형 문제 해결 (XOR, Moons, Circles)
5. 은닉층 크기의 영향
6. 미니배치 학습

### 핵심 개념:
- **순전파**: 입력에서 출력까지 신호가 전달되는 과정
- **역전파**: 오차를 이용해 가중치를 업데이트하는 과정
- **활성화 함수**: 비선형성을 추가하여 복잡한 패턴 학습 가능
- **경사하강법**: 손실함수를 최소화하는 최적화 알고리즘

다음 단계에서는 PyTorch를 사용하여 더 효율적으로 신경망을 구현하는 방법을 배워보겠습니다!